In [6]:
import torch
x = torch.Tensor([[1, 2],[3, 4]])
print(x)
x.size()

tensor([[1., 2.],
        [3., 4.]])


torch.Size([2, 2])

In [95]:
x = torch.rand(1, 1, 2, 3)
print(x)
patch_size = 3
n, c, h, w = x.size()
step = patch_size//2
print(step)
y = torch.zeros(n, c, h + 2*step, w + 2*step)
y[:, :, step:step+h, step:step+w] = x
y[:, :, step:step+h,:step] = x[:, :, :,0:1].expand((n, c, h,step))
y[:, :, step:step+h,step+w:step+2*w] = x[:, :, :,-1:w+2*step].expand((n, c, h,step))
y[:, :, :step, :] = y[:, :, step:step+1,:].expand((n, c, step, w+2*step))
y[:, :, h+step:, :] = y[:, :, h+step-1:h+step,:].expand((n, c, step, w+2*step))
print(y)

tensor([[[[0.0759, 0.4449, 0.9818],
          [0.9194, 0.8485, 0.7078]]]])
1
tensor([[[[0.0759, 0.0759, 0.4449, 0.9818, 0.9818],
          [0.0759, 0.0759, 0.4449, 0.9818, 0.9818],
          [0.9194, 0.9194, 0.8485, 0.7078, 0.7078],
          [0.9194, 0.9194, 0.8485, 0.7078, 0.7078]]]])


In [96]:
ex_feat = torch.zeros(n, c, patch_size*patch_size, h, w) #extended feature map
# build feat tensor for flann
for i in range(patch_size):
    for j in range(patch_size):
        pn = i * patch_size + j
        ex_feat[:, :, pn, :, :] = y[:, :, i:i+h, j:j+w]
ex_feat

tensor([[[[[0.0759, 0.0759, 0.4449],
           [0.0759, 0.0759, 0.4449]],

          [[0.0759, 0.4449, 0.9818],
           [0.0759, 0.4449, 0.9818]],

          [[0.4449, 0.9818, 0.9818],
           [0.4449, 0.9818, 0.9818]],

          [[0.0759, 0.0759, 0.4449],
           [0.9194, 0.9194, 0.8485]],

          [[0.0759, 0.4449, 0.9818],
           [0.9194, 0.8485, 0.7078]],

          [[0.4449, 0.9818, 0.9818],
           [0.8485, 0.7078, 0.7078]],

          [[0.9194, 0.9194, 0.8485],
           [0.9194, 0.9194, 0.8485]],

          [[0.9194, 0.8485, 0.7078],
           [0.9194, 0.8485, 0.7078]],

          [[0.8485, 0.7078, 0.7078],
           [0.8485, 0.7078, 0.7078]]]]])

In [97]:
z1 = ex_feat.reshape((n, c * patch_size * patch_size, h, w))
z2 = z1.reshape((n, c*patch_size*patch_size, h*w))
print(z1)
print(z2)

tensor([[[[0.0759, 0.0759, 0.4449],
          [0.0759, 0.0759, 0.4449]],

         [[0.0759, 0.4449, 0.9818],
          [0.0759, 0.4449, 0.9818]],

         [[0.4449, 0.9818, 0.9818],
          [0.4449, 0.9818, 0.9818]],

         [[0.0759, 0.0759, 0.4449],
          [0.9194, 0.9194, 0.8485]],

         [[0.0759, 0.4449, 0.9818],
          [0.9194, 0.8485, 0.7078]],

         [[0.4449, 0.9818, 0.9818],
          [0.8485, 0.7078, 0.7078]],

         [[0.9194, 0.9194, 0.8485],
          [0.9194, 0.9194, 0.8485]],

         [[0.9194, 0.8485, 0.7078],
          [0.9194, 0.8485, 0.7078]],

         [[0.8485, 0.7078, 0.7078],
          [0.8485, 0.7078, 0.7078]]]])
tensor([[[0.0759, 0.0759, 0.4449, 0.0759, 0.0759, 0.4449],
         [0.0759, 0.4449, 0.9818, 0.0759, 0.4449, 0.9818],
         [0.4449, 0.9818, 0.9818, 0.4449, 0.9818, 0.9818],
         [0.0759, 0.0759, 0.4449, 0.9194, 0.9194, 0.8485],
         [0.0759, 0.4449, 0.9818, 0.9194, 0.8485, 0.7078],
         [0.4449, 0.9818, 0.9818, 0.84

In [66]:
torch.zeros((n, 2, h, w), dtype=int)

tensor([[[[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]],

         [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]]])

In [101]:
import numpy as np

a = np.arange(6)
a

array([0, 1, 2, 3, 4, 5])

In [104]:
a.reshape(2, 3)

array([[0, 1, 2],
       [3, 4, 5]])

In [107]:
print(a//2)
print(a%2)

[0 0 1 1 2 2]
[0 1 0 1 0 1]
